# Install the required packages

In [81]:
import sys
import os
import glob
import matplotlib.pyplot as plt

import numpy as np
#to work with text files
import pandas as pd 

#pre-processing imports
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords') # stopword 
nltk.download('punkt') # word_tokenize

#imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# word2vec installation

In [11]:
!brew install wget
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
data_path= "DATAPATH"
%time 
w2v_model = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)
print('done loading Word2Vec')

/bin/bash: brew: command not found
--2021-11-20 20:41:43--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.12.86
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.12.86|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs
done loading Word2Vec


In [14]:
# Inspect the model
word2vec_vocab = w2v_model.vocab.keys()
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]
print(len(word2vec_vocab))
print(word2vec_vocab_lower[500:1000])

3000000
['doing', 'face', 'low', 'higher', 'site', 'once', 'yet', 'hours', 'america', 'control', 'received', 'rate', 'career', 'bush', 'teams', 'known', 'offer', 'race', 'ever', 'experience', 'playing', 'name', 'possible', 'countries', 'mr.', 'average', 'together', 'using', '9', 'cut', 'while', 'total', 'round', 'young', 'nearly', 'shares', 'member', 'campaign', 'media', 'needs', 'why', 'house', 'issues', 'costs', 'fire', '##-#', 'victory', 'player', 'began', 'sure', 'story', 'per_cent', 'north', 'his', 'staff', 'order', 'war', 'large', 'interest', 'stock', 'food', 'research', 'key', 'india', 'south', 'morning', 'conference', 'senior', 'global', 'center', 'death', 'person', 'thought', 'gave', 'feel', 'energy', 'history', 'recently', 'largest', 'no.', 'general', 'official', 'released', 'wanted', 'meet', 'short', 'outside', 'running', 'live', 'ball', 'online', 'real', 'position', 'fact', 'fell', 'nine', 'december', 'front', 'action', 'defense', 'problem', 'problems', 'mr', 'nation', 'nee

In [15]:
# Ref: https://www.geeksforgeeks.org/python-os-chdir-method/
# Ref: https://stackoverflow.com/questions/9234560/find-all-csv-files-in-a-directory-using-python/12280052 

def get_list_filenames(cwd, path, extension):
    try:
        os.chdir(path)
        print("Successfully change the directory!")
        filenames = glob.glob('*.{}'.format(extension))
        print("Number of files: ", len(filenames))

    # Caching the exception
    except: 
        print("Something wrong with specified directory. Exception- ", sys.exc_info()) 

    # handling with finally: restore the path which is the current directory before changing directory           
    finally: 
        print("Restoring the path") 
        os.chdir(cwd) # Change it back to cwd (original directory)
        print("Current directory is-", os.getcwd()) 
        
    return filenames


# Ref: https://realpython.com/python-keras-text-classification/

# Get a list of dataframe after reading all csv files given a path
def get_all_image_files_read(path, filenames):
    list_img = []
    for file in filenames:
        img = imageio.imread(path+file)
        list_img.append(img)
    return list_img

In [16]:
%cd /content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
!pwd

/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
/content/drive/MyDrive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake


In [80]:
# initial directory 
cwd = os.getcwd()
path_org = './' # the directory where all the output result are located.
extension = 'txt'

fake_textfiles = get_list_filenames(cwd, path_org, extension)
print(fake_textfiles[0])
print(len(fake_textfiles))

Successfully change the directory!
Number of files:  4500
Restoring the path
Current directory is- /content/drive/My Drive/American_University/2021_Fall/DATA-642-001_Advanced Machine Learning/data/data_ready/texts/fake
FAKE_BCD3501.txt
4500


- 360 from B fake videos
- 1884 from C fake videos
- 2256 from D fake videos

Totally 4500 text files

In [18]:
print(fake_textfiles[2300])

FAKE_BCD1801.txt


# Tokenize and Remove Stopwords
- Keeping punctuations could be relevant for this task

In [79]:
aaa = []
aaa.append(open(fake_textfiles[2300]).readline())
print(aaa)

mystopwords = set(stopwords.words("english"))
# function to tokenize tweets, remove stopwords and numbers.
#k eeping punctuations and emoticon symbols could be relevant for this task!
def preprocess_corpus(texts):
  def remove_stops_digits(tokens):
  #Nested function that removes stopwords and digits from a list of tokens
    return [token for token in tokens if token not in mystopwords and not token.isdigit()]
    #This return statement below uses the above function to process twitter tokenizer output further.
  return [remove_stops_digits(word_tokenize(content)) for content in texts]


bbb = preprocess_corpus(aaa)
flatten_list = sum(bbb, [])
print(flatten_list)


["to the point where okay I'm going to go this way and you go this way because the camera has this this way in"]
['point', 'okay', 'I', "'m", 'going', 'go', 'way', 'go', 'way', 'camera', 'way']
point  :  point
okay  :  okay
I  :  I
'm  :  'm
going  :  go
go  :  go
way  :  way
go  :  go
way  :  way
camera  :  camera
way  :  way
['point', 'okay', 'I', "'m", 'going', 'go', 'way', 'go', 'way', 'camera', 'way']


# Create **word2vec** Vetorzation Function

In [88]:
# Creating a feature vector by averaging all embeddings for all sentences
def embedding_feats(list_of_lists):
  DIMENSION = 300
  zero_vector = np.zeros(DIMENSION)
  feats = []
  for tokens in list_of_lists:
    feat_for_this = np.zeros(DIMENSION)
    count_for_this = 0
    for token in tokens:
      if token in w2v_model:
        feat_for_this += w2v_model[token]
        count_for_this +=1
    feats.append(feat_for_this/count_for_this)
  return feats

"""
train_vector = embedding_feats(flatten_list)
train_vector = np.array(train_vector)
print(train_vector.shape)
print(train_vector)

flaa = np.ndarray.flatten(train_vector)
print(flaa.shape)
print(flaa)
"""

'\ntrain_vector = embedding_feats(flatten_list)\ntrain_vector = np.array(train_vector)\nprint(train_vector.shape)\nprint(train_vector)\n\nflaa = np.ndarray.flatten(train_vector)\nprint(flaa.shape)\nprint(flaa)\n'

# create `FAKE_RTVC_B` text features
- total frames: 360 videos * 4 frames = 1440

In [98]:
fake_B = []
for i in range(0, 360):
  tmp = []
  tmp.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=object)
  text_vectors = np.ndarray.flatten(text_vectors)
  print(text_vectors.shape)
  del tmp
  fake_B.append(text_vectors)

(300,)
(3600,)
(300,)
(300,)
(1500,)
(300,)
(3600,)
(2400,)
(300,)
(2400,)
(1200,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(5400,)
(5400,)
(5400,)
(1500,)
(3000,)
(3000,)
(1500,)
(3000,)
(3000,)
(3000,)
(5400,)
(1500,)
(1500,)
(1500,)
(5400,)
(3000,)
(1500,)
(1500,)
(3000,)
(1500,)
(5400,)
(3000,)
(5400,)
(5400,)
(3000,)
(3900,)
(1500,)
(1500,)
(3000,)
(5400,)
(5400,)
(1200,)
(2100,)
(3900,)
(6900,)
(3900,)
(6900,)
(6900,)
(6900,)
(6900,)
(3900,)
(6900,)
(3900,)
(3900,)
(6900,)
(6900,)
(3900,)
(6900,)
(3900,)
(3900,)
(6900,)
(3900,)
(3900,)
(1500,)
(300,)
(2400,)
(3900,)
(2400,)
(2400,)
(2400,)
(2400,)
(2400,)
(2400,)
(2400,)
(2400,)
(2400,)
(2400,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(3000,)
(3000,)
(1200,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(6000,)
(1200,)
(1500,)
(3000,)
(3000,)
(3000,)
(3000,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(5400,)
(2700,)
(2700,)
(5400,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2700,)
(2100,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(4200,)
(3000,)
(4200,)
(4200,)
(4200,)
(4200,)
(4200,)
(4200,)
(4200,)
(4200,)
(4200,)
(5400,)
(5400,)
(5400,)
(5400,)
(5400,)
(5400,)
(5400,)
(5400,)
(5400,)
(5400,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(900,)
(300,)
(900,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(2100,)
(1200,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(6300,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(5700,)
(300,)
(5700,)
(6300,)
(6300,)
(6300,)
(300,)
(300,)
(6300,)
(300,)
(300,)
(300,)
(6300,)
(300,)
(300,)
(6300,)
(300,)
(6300,)
(300,)
(6300,)
(6300,)
(6300,)
(2700,)
(300,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(1800,)
(3300,)
(3600,)
(1200,)
(3000,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)
(2100,)

In [99]:
print(len(fake_B))
#print(fake_B[100])

# Because we capture 4 frame in fake B set, so we need to duplicate the text 4 times 
B = np.concatenate((fake_B, fake_B, fake_B, fake_B), axis = 0)
print(len(B))

360
1440


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# Create `FAKE_FSGAN_C` to text features
- total frames: 1884 videos * 2 frames = 3768

In [ ]:
fake_C = []
for i in range(360, 2244):
  tmp = []
  tmp.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=object)
  text_vectors = np.ndarray.flatten(text_vectors)
  print(text_vectors.shape)
  del tmp
  fake_C.append(text_vectors)

(1500,)
(1500,)
(900,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(900,)
(2400,)
(1500,)
(1500,)
(2400,)
(3600,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(3000,)
(1800,)
(3000,)
(3000,)
(600,)
(6300,)
(4800,)
(1500,)
(5100,)
(5100,)
(1500,)
(1500,)
(4800,)
(4800,)
(1500,)
(4800,)
(1500,)
(4800,)
(1500,)
(1500,)
(1500,)
(1500,)
(1500,)
(4800,)
(4800,)
(4800,)
(2400,)
(2400,)
(2400,)
(2100,)
(2400,)
(2400,)
(2100,)
(2400,)
(2400,)
(2400,)
(3300,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


(2400,)
(3900,)
(3300,)
(8100,)
(2400,)
(3900,)
(2400,)
(600,)
(600,)
(3900,)
(600,)
(9900,)
(9900,)
(2100,)
(9900,)
(1500,)
(3900,)
(1500,)
(1500,)
(9900,)
(9900,)
(1500,)
(600,)
(600,)
(2100,)
(5700,)
(9900,)
(600,)
(600,)
(5700,)
(5700,)
(1500,)
(3300,)
(3300,)
(3900,)
(1200,)
(9900,)
(3900,)
(600,)
(600,)
(9900,)
(600,)
(600,)
(600,)
(2400,)
(3900,)
(3900,)
(3900,)
(2400,)
(600,)
(600,)
(3300,)
(2400,)
(3900,)
(3900,)
(5700,)
(5700,)
(3900,)
(3300,)
(3900,)
(3300,)
(9900,)
(3900,)
(2400,)
(3900,)
(2400,)
(2400,)
(2400,)
(9900,)
(2100,)
(3900,)
(2400,)
(2400,)
(2100,)
(3300,)
(2100,)
(3300,)
(3300,)
(3300,)
(3900,)
(3300,)
(900,)
(2100,)
(2100,)
(1500,)
(5100,)
(3600,)
(3300,)
(3000,)
(300,)
(2100,)
(3300,)
(3600,)
(3300,)
(2400,)
(900,)
(7800,)
(2100,)
(1800,)
(2100,)
(2100,)
(4500,)
(2100,)
(2400,)
(4200,)
(5400,)
(1200,)
(3600,)
(3000,)
(1500,)
(900,)
(2700,)
(1200,)
(3900,)
(2400,)
(3300,)
(3000,)
(300,)
(5100,)
(2700,)
(2400,)
(2400,)
(4200,)
(1800,)
(2400,)
(2400,)
(1800,)
(21

In [ ]:
print(len(fake_C))
#print(fake_B[100])

# Because we capture 2 frame in fake C set , so we need to duplicate the text 2 times 
C = np.concatenate((fake_C, fake_C), axis = 0)
print(len(C))

# Create `FAKE_FACESWAP_D_W2L` to text features
- total frames: 2256 videos * 2 frames = 4512

In [ ]:
fake_D = []
for i in range(2244, len(fake_textfiles)):
  tmp = []
  tmp.append(open(fake_textfiles[i]).readline())
  # apply to function
  tmp = preprocess_corpus(tmp)
  # faltten 2d list to 1d
  tmp = sum(tmp, [])
  text_vectors = embedding_feats(tmp)
  text_vectors = np.array(text_vectors, dtype=object)
  text_vectors = np.ndarray.flatten(text_vectors)
  print(text_vectors.shape)
  del tmp
  fake_D.append(text_vectors)

In [ ]:
print(len(fake_D))
#print(fake_B[100])

# Because we capture 2 frame in fake C set , so we need to duplicate the text 2 times 
D = np.concatenate((fake_D, fake_D), axis = 0)
print(len(D))

# References
- https://code.google.com/archive/p/word2vec/